In [ ]:
from ctgan import CTGAN
from ct_pool_match import load_tabular_data, train
from ctgan import load_demo
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import torch
import pandas as pd
import numpy as np


In [ ]:
import tensorflow as tf
print(torch.cuda.is_available())

In [ ]:
real_data = pd.read_csv("iotid20.csv")

In [ ]:
scaler = StandardScaler()
label = LabelEncoder()
# data[data.columns[:-1]] = scaler.fit_transform(data[data.columns[:-1]])
real_data[real_data.columns[:-1]] = np.array(real_data[real_data.columns[:-1]], dtype=float)
real_data.replace([np.inf, -np.inf], np.nan, inplace=True)
#data.iloc[:, -1] = label.fit_transform(data.iloc[:, -1])
real_data.dropna(inplace=True)
real_data.head(3).T

In [ ]:

def get_discrete_columns(dataframe, max_unique_values=80):
    """
    Hàm để xác định các cột rời rạc (discrete), bao gồm các cột có số lượng giá trị unique ít hơn hoặc bằng max_unique_values.
    Ngoài ra, có thể chuyển đổi kiểu dữ liệu của các cột rời rạc sang string.
    
    - dataframe: DataFrame chứa dữ liệu
    - max_unique_values: Số lượng giá trị unique tối đa để coi là discrete (mặc định là 80)
    - convert_to_string: Nếu True, chuyển kiểu dữ liệu của các cột rời rạc sang string.
    
    Trả về:
    - discrete_columns: Danh sách tên các cột rời rạc.
    - discrete_indices: Danh sách chỉ số (index) của các cột rời rạc.
    """
    discrete_columns = []
    discrete_indices = []
    
    for col in dataframe.columns:
        unique_values = dataframe[col].nunique()
        print(f"Column '{col}' - Number of unique values: {unique_values}")
        # Chọn các cột có kiểu dữ liệu số và số lượng giá trị unique nhỏ hơn hoặc bằng max_unique_values
        if dataframe[col].dtype in ['float64', 'int64'] and unique_values <= max_unique_values:
            discrete_columns.append(col)
            discrete_indices.append(dataframe.columns.get_loc(col))
            
            dataframe[col] = dataframe[col].astype(str)
            print(f"Column '{col}' converted to string.")
    
    return discrete_columns, discrete_indices 


# Ví dụ với dataset của bạn
discrete_columns, discrete_indices = get_discrete_columns(real_data)
discrete_columns.append("Target")
print("Discrete Columns:", discrete_columns)
print("Indices of Discrete Columns:", discrete_indices)

In [ ]:
ctgan = CTGAN()

In [ ]:
ctgan.load_model("output\ctgan_model_epoch_20.pth")
#ctgan.fit(real_data, discrete_columns)

In [ ]:
def get_unique_labels(dataframe, label_column):
    """
    Trả về mảng các giá trị nhãn duy nhất từ một cột trong tập dữ liệu.

    Args:
        dataframe (pd.DataFrame): Tập dữ liệu đầu vào.
        label_column (str): Tên cột chứa nhãn.

    Returns:
        list: Danh sách các nhãn duy nhất.
    """
    if label_column not in dataframe.columns:
        raise ValueError(f"Cột '{label_column}' không tồn tại trong tập dữ liệu.")
    
    unique_labels = dataframe[label_column].unique()
    return unique_labels.tolist()

In [ ]:
unique_labels = get_unique_labels(real_data, 'Target')
unique_labels

In [ ]:
def sample_by_label(ctgan, real_data, label_column):
    """
    Tạo dữ liệu tổng hợp bằng cách sinh một số lượng mẫu bằng với số lượng mẫu của từng nhãn trong dữ liệu thực.

    Args:
        ctgan (CTGAN): Đối tượng CTGAN đã được huấn luyện.
        real_data (pd.DataFrame): Tập dữ liệu thực.
        label_column (str): Tên cột chứa nhãn.

    Returns:
        pd.DataFrame: Dữ liệu tổng hợp với số lượng mẫu tương ứng cho từng nhãn.
    """
    # Kiểm tra nếu cột nhãn tồn tại
    if label_column not in real_data.columns:
        raise ValueError(f"Cột '{label_column}' không tồn tại trong tập dữ liệu.")
    
    # Lấy danh sách các nhãn duy nhất và số lượng mẫu của từng nhãn
    label_counts = real_data[label_column].value_counts()
    synthetic_data = []

    # Sinh dữ liệu cho từng nhãn
    for label, count in label_counts.items():
        print(f"Sinh {count} mẫu cho nhãn: {label}")
        batch_size = ctgan._batch_size  # Sử dụng batch_size từ CTGAN
        total_synthetic = 0

        current_synthetic = []
        while total_synthetic < count:
            remaining = min(batch_size, count - total_synthetic)
            synthetic_samples = ctgan.sample_by_label(
                n=remaining, 
                label_column=label_column, 
                label_value=label
            )
            synthetic_samples_df = pd.DataFrame(synthetic_samples)
            current_synthetic.append(synthetic_samples_df)
            total_synthetic += len(synthetic_samples_df)
        
        # Kết hợp các mẫu tổng hợp cho nhãn này
        current_synthetic = pd.concat(current_synthetic, ignore_index=True)
        
        # Kiểm tra và điều chỉnh số lượng mẫu
        if total_synthetic > count:
            # Nếu nhiều hơn, cắt bớt
            current_synthetic = current_synthetic.iloc[:count]
            print(f"Đã cắt bớt {total_synthetic - count} mẫu cho nhãn: {label}")
        elif total_synthetic < count:
            # Nếu ít hơn, sinh thêm mẫu
            print(f"Thiếu {count - total_synthetic} mẫu, tiếp tục sinh...")
            remaining = count - total_synthetic
            additional_samples = ctgan.sample_by_label(
                n=remaining, 
                label_column=label_column, 
                label_value=label
            )
            additional_samples_df = pd.DataFrame(additional_samples)
            current_synthetic = pd.concat([current_synthetic, additional_samples_df], ignore_index=True)
        
        # Thêm dữ liệu của nhãn này vào danh sách tổng hợp
        synthetic_data.append(current_synthetic)
        print(f"Hoàn thành {len(current_synthetic)} mẫu cho nhãn: {label}")
    
    # Kết hợp tất cả các mẫu tổng hợp
    synthetic_data = pd.concat(synthetic_data, ignore_index=True)
    return synthetic_data


In [ ]:
def print_class_distribution(real_data, synthetic_data, label_column):
    """
    In ra số lượng mẫu của từng class (nhãn) trong cả tập gốc và tập sinh.

    Args:
        real_data (pd.DataFrame): Tập dữ liệu thực.
        synthetic_data (pd.DataFrame): Tập dữ liệu sinh.
        label_column (str): Tên cột chứa nhãn.
    """
    # Đếm số lượng mẫu của từng nhãn trong tập gốc
    real_counts = real_data[label_column].value_counts()

    # Đếm số lượng mẫu của từng nhãn trong tập sinh
    synthetic_counts = synthetic_data[label_column].value_counts()

    print("Số lượng mẫu của từng class:")
    print("-" * 40)
    print(f"{'Class':<15}{'Real Count':<15}{'Synthetic Count'}")
    print("-" * 40)

    # Lấy danh sách tất cả các nhãn (cả trong tập gốc và tập sinh)
    all_classes = set(real_counts.index).union(set(synthetic_counts.index))

    # In ra số lượng mẫu cho từng nhãn
    for cls in sorted(all_classes):
        real_count = real_counts.get(cls, 0)  # Lấy số lượng mẫu từ tập gốc
        synthetic_count = synthetic_counts.get(cls, 0)  # Lấy số lượng mẫu từ tập sinh
        print(f"{cls:<15}{real_count:<15}{synthetic_count}")


In [ ]:
# Create synthetic data
##synthetic_data = sample_by_label(ctgan=ctgan, real_data=real_data, label_column='Target')

In [ ]:
#print_class_distribution(real_data=real_data, synthetic_data=synthetic_data, label_column='Target')

In [ ]:
#synthetic_data.to_csv("ctgan_80_same_samplepclass.csv", index=False)

In [ ]:
def preprocess_data(data):
    """
    Tiền xử lý dữ liệu bằng cách chuẩn hóa các cột số và mã hóa các cột danh mục.
    
    Args:
        data (pd.DataFrame): Dữ liệu đầu vào.
    
    Returns:
        tuple: 
            - pd.DataFrame: Dữ liệu đã được tiền xử lý.
            - LabelEncoder: Bộ mã hóa nhãn được áp dụng.
            - dict: Ánh xạ giữa nhãn gốc và mã hóa.
    """
    data = data.copy()  # Tạo bản sao để tránh thay đổi dữ liệu gốc
    
    # Lấy danh sách các cột numeric và categorical
    numerical_columns = data.select_dtypes(include=['float64', 'int64']).columns
    categorical_columns = data.select_dtypes(include=['object', 'category']).columns

    # Chuẩn hóa các cột số
    if len(numerical_columns) > 0:
        scaler = StandardScaler()
        data[numerical_columns] = scaler.fit_transform(data[numerical_columns])
    
    encoder = LabelEncoder()
    label_mapping = {}  # Lưu ánh xạ nhãn gốc -> mã hóa
    # Mã hóa các cột danh mục
    if len(categorical_columns) > 0:
        for col in categorical_columns:
            data[col] = encoder.fit_transform(data[col])
            # Tạo ánh xạ nhãn gốc -> mã hóa cho cột nhãn
            if col == 'Target':  # Giả định cột nhãn tên là 'Target'
                label_mapping = dict(zip(encoder.classes_, range(len(encoder.classes_))))
                label_mapping = {v: k for k, v in label_mapping.items()}
    return data, label_mapping


In [ ]:
import os
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset
class Args:
    batch_size = 256
    epochs = 10
    print_freq = 10
    data_path = 'your_data.csv'  # Đường dẫn tới dữ liệu
    output_dir = './models'
    generator_lr = 2e-4

if not os.path.exists(Args.output_dir):
    os.makedirs(Args.output_dir)

real_traindata, label_mapping = preprocess_data(real_data)
# Load dữ liệu

real_traindata

In [ ]:
label_mapping

In [ ]:
# Kiểm tra cột target và mã hóa
print(real_traindata[real_traindata.columns[-1]].head())  # Giá trị ban đầu
print(real_traindata['Target'].head())      # Giá trị sau mã hóa


In [ ]:
train_data = TensorDataset(torch.tensor(real_traindata.values, dtype=torch.float32))
trainloader = DataLoader(train_data, batch_size=Args.batch_size, shuffle=True, drop_last=True)

In [ ]:
discriminator = ctgan.getDiscriminator(real_traindata.values,1)

In [ ]:
import torch.nn as nn
# Khởi tạo optimizer
optim_d = torch.optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))
criterion = nn.BCEWithLogitsLoss() # type: ignore

# Huấn luyện discriminator với dữ liệu từ CTGAN
train(Args, ctgan, discriminator, optim_d, trainloader, label_mapping, criterion)